## Classifying with  _sklearn.linear_model.LinearRegression_

Using linear regression to predict most prone ages to sofer violence from line 144 data source

### Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, top_k_accuracy_score

### Load from datasource

**Please, execute 01_pre_process.ipynb before**

In [3]:
data = pd.read_parquet('./data/linea_144_processed.parquet')

assert data.shape == (84861, 19)
assert data.fecha.dtype == 'datetime64[ns]', "'fecha' must be a datetime64 type"

### Labeled data

In [4]:
fields_to_encode = data.columns.drop(['fecha', 'edad_persona_en_situacion_de_violencia'])

for field in fields_to_encode:
    le = LabelEncoder()
    le.fit(data[field])
    data[field] = le.transform(data[field])

### Training model

In [34]:
# Features available/candidates 

"""
['fecha', 'prov_residencia_persona_en_situacion_violencia',
'genero_persona_en_situacion_de_violencia',
'edad_persona_en_situacion_de_violencia',
'pais_nacimiento_persona_en_situacion_de_violencia',
'tipo_de_violencia_fisica', 'tipo_de_violencia_psicologica',
'tipo_de_violencia_sexual', 'tipo_de_violencia_economica_y_patrimonial',
'tipo_de_violencia_simbolica', 'tipo_de_violencia_domestica',
'modalidad_de_violencia_institucional',
'modalidad_de_violencia_laboral',
'modalidad_violencia_contra_libertad_reproductiva',
'modalidad_de_violencia_obstetrica', 'modalidad_de_violencia_mediatica',
'modalidad_de_violencia_otras', 'vinculo_con_la_persona_agresora',
'genero_de_la_persona_agresora']
"""

features = ['tipo_de_violencia_fisica', 'tipo_de_violencia_psicologica', 'tipo_de_violencia_sexual', 
            'tipo_de_violencia_economica_y_patrimonial', 'tipo_de_violencia_simbolica', 
            'tipo_de_violencia_domestica', 'modalidad_de_violencia_institucional', 
            'modalidad_de_violencia_laboral', 'modalidad_violencia_contra_libertad_reproductiva', 
            'modalidad_de_violencia_obstetrica', 'modalidad_de_violencia_mediatica', 'modalidad_de_violencia_otras']

def get_linear_regression_to(_feature):
    
    # feature: type of violence
    X = np.array(data[_feature].values).reshape(-1,1)

    # Target: age
    y = data['edad_persona_en_situacion_de_violencia'].values

    # Test
    assert len(y) == len(X), "fatures and target must be equals"

    """
    EXPLANATION OF VARIABLES
    ===================

    X_train = features to train/fit the model
    y_train = targets to train/fit the model
    The two vars above are used to train/fit the model, usually fit(X_train, y_train)
    ---- ---- ---- ---- 
    X_test = features to test the model
    y_test = targets to test the model
    The two vars above are used to test the accuracy, usually running a prediction function: predict(X_text) 
    and comparing the accuracy with y_test
    """

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

    # Processing arrays to integer
    X_train, X_test, y_train, y_test = [np.array(_arr).astype(int) for _arr in [X_train, X_test, y_train, y_test]]

    # Running model
    linear_regression = LinearRegression()
    linear_regression.fit(X_train, y_train)

    # Get predictions
    predicted = linear_regression.predict(X_test)
    
    # running accuracy_score: Classification metrics can't handle a mix of continuous and multiclass targets
    # https://scikit-learn.org/stable/modules/model_evaluation.html
    # accuracy = accuracy_score(predicted, y_test)
    # accuracy = balanced_accuracy_score(y_test, predicted), same problem
    # accuracy = top_k_accuracy_score(y_train, predicted) 

    predicted = np.array(predicted).astype(int)
    accuracy = accuracy_score(y_test, predicted)

    return (_feature, round(accuracy,5))

[get_linear_regression_to(feature) for feature in features]


[('tipo_de_violencia_fisica', 0.14205),
 ('tipo_de_violencia_psicologica', 0.02581),
 ('tipo_de_violencia_sexual', 0.02557),
 ('tipo_de_violencia_economica_y_patrimonial', 0.1556),
 ('tipo_de_violencia_simbolica', 0.16379),
 ('tipo_de_violencia_domestica', 0.04961),
 ('modalidad_de_violencia_institucional', 0.02522),
 ('modalidad_de_violencia_laboral', 0.02516),
 ('modalidad_violencia_contra_libertad_reproductiva', 0.02533),
 ('modalidad_de_violencia_obstetrica', 0.02522),
 ('modalidad_de_violencia_mediatica', 0.02516),
 ('modalidad_de_violencia_otras', 0.03099)]